# How to build an AI Agent with Upsonic and the ClickHouse MCP Server

## Install libraries

In [ ]:
!pip install -q --upgrade pip

In [ ]:
!pip install -q upsonic asyncpg aiosqlite

## Setup credentials
Let's provide our OpenAI API key.

In [9]:
import os, getpass

In [10]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key:")

Enter OpenAI API Key: ········


## ClickHouse MCP Server

In [17]:
class DatabaseMCP:
    """
    MCP server for ClickHouse database operations.
    Provides tools for querying tables and databases
    """
    command="uv"
    args=[
        "run",
        "--with",
        "mcp-clickhouse",
        "--python",
        "3.10",
        "mcp-clickhouse"
    ]
    env={
        "CLICKHOUSE_HOST": "sql-clickhouse.clickhouse.com",
        "CLICKHOUSE_PORT": "8443",
        "CLICKHOUSE_USER": "demo",
        "CLICKHOUSE_PASSWORD": "",
        "CLICKHOUSE_SECURE": "true",
        "CLICKHOUSE_VERIFY": "true",
        "CLICKHOUSE_CONNECT_TIMEOUT": "30",
        "CLICKHOUSE_SEND_RECEIVE_TIMEOUT": "30"
    }


## Initialize agent and task

In [12]:
from upsonic import Agent, Task

/Users/markhneedham/.local/share/uv/python/cpython-3.12.0-macos-aarch64-none/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
database_agent = Agent(
    name="Data Analyst",
    role="ClickHouse specialist.",
    goal="Query ClickHouse database and tables and answer questions",
    model="openai/o3-mini"
)

In [18]:
task = Task(
    description="Tell me what happened in the UK property market in the 2020s. Use ClickHouse to answer this question.",
    tools=[DatabaseMCP]
)

## Run agent

In [19]:
# Execute the workflow
workflow_result = database_agent.do(task)
print("\nMCP Workflow Result:")
print(workflow_result)

╭───────────────────────── 🤖 Agent Started ─────────────────────────╮
│  Agent Status:             🚀 Started to work                      │
│  Agent Name:               Data Analyst                            │
╰────────────────────────────────────────────────────────────────────╯

Task exception was never retrieved
future: <Task finished name='Task-54' coro=<AsyncClient.aclose() done, defined at /Users/markhneedham/.cache/uv/archive-v0/3LtP5Av0JJE9Y2Ao_exK0/lib/python3.12/site-packages/httpx/_client.py:1978> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/Users/markhneedham/.cache/uv/archive-v0/3LtP5Av0JJE9Y2Ao_exK0/lib/python3.12/site-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/Users/markhneedham/.cache/uv/archive-v0/3LtP5Av0JJE9Y2Ao_exK0/lib/python3.12/site-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/Users/markhneedham/.cache/uv/archive-v0/3LtP5Av0JJE9Y2Ao_exK0/lib/python3.12/site-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/Users/markhneedham/.cache/uv/archive-v0/3LtP5Av0JJE9Y2Ao_exK0/lib/python3.12/site-packages/httpcore/_async/c

╭─────────────────── Tool Usage Summary (4 tools) ───────────────────╮
│  Tool Name         Parameters           Result                     │
│  list_databases    {}                   ['amazon',                 │
│                                         'bluesky',                 │
│                                         'country', 'covid',        │
│                                         'defa...                   │
│  list_tables       {"database": "uk",   []                         │
│                    "like": "property"}                             │
│  list_tables       {"database": "uk"}   [{'database': 'uk',        │
│                                         'name': 'uk_codes',        │
│                                         'engine...                 │
│  run_select_query  {"query": "SELECT    {'columns':                │
│                    toYear(date) AS      ['year',                   │
│                    year, count()...     'num_transactions',        │
│                                         'avg_p...                  │
╰────────────────────────────────────────────────────────────────────╯

╭─────────────────────────── Task Result ────────────────────────────╮
│  Result:  Based on data from the uk_price_paid table for           │
│           the period starting January 2020, here are some          │
│           key observations:                                        │
│                                                                    │
│           • In 2020 there were about 896K transactions with        │
│           an average sale price of around £377K.                   │
│           • In 2021 the market activity picked up to               │
│           roughly 1.28M transactions and the average price         │
│           increased to about £389K. Notably, some                  │
│           properties fetched very high pric[bold                   │
│           white]...[/bold white]                                   │
╰────────────────────────────────────────────────────────────────────╯

╭─────────────────────────── Task Metrics ───────────────────────────╮
│  Price ID:              468cfbf2-4fd4-4ecc-b3f4-3311eb327f…        │
│                                                                    │
│  Input Tokens:          13,774                                     │
│  Output Tokens:         2,032                                      │
│  Total Estimated Cost:  $0.0241                                    │
│  Time Taken:            39.11 seconds                              │
╰────────────────────────────────────────────────────────────────────╯


MCP Workflow Result:
Based on data from the uk_price_paid table for the period starting January 2020, here are some key observations:

• In 2020 there were about 896K transactions with an average sale price of around £377K.  
• In 2021 the market activity picked up to roughly 1.28M transactions and the average price increased to about £389K. Notably, some properties fetched very high prices – the maximum reached around £900M.  
• In 2022, the average price further increased to approximately £414K with over 1.07M transactions, although the top-end price dropped to about £480M from the previous year’s peak.  
• For 2023, the transaction volume decreased to around 855K and the average price dropped to nearly £405K; the maximum price also fell further to roughly £268M.  
• In 2024, while the transactions stayed at about 860K, the average price declined to about £391K with the highest price seen being around £164M.  
• In 2025, there was a sharper decline in volume (around 377K transaction